In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import time
import urllib.request
import urllib.error

player_data_list = []
count = 0
has_data = False
header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36 Edg/90.0.818.66"
}
seasons = range(2015, 2022, 1)


def check_internet():
    try:
        header = {"pragma": "no-cache"}
        req = urllib.request.Request("http://www.google.ro", headers=header)
        urllib.request.urlopen(req, timeout=2)
        return True
    except urllib.error.URLError:
        return False

has_data=False
header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36 Edg/90.0.818.66"
}
player_data_list = []
count = 0
while True:
    conn = check_internet()
    if conn:
        try:
            for season in seasons:
                comps = requests.get(
                    f"https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1/plus/?saison_id={season}",
                    headers=header)
                comps_Soup = BeautifulSoup(comps.content, "html.parser")
                teams = comps_Soup.select('#yw1 .no-border-links a:nth-child(1)')
                for team in teams:
                    team_name = team['href'].split('/')[1]
                    team_id = team['href'].split('/')[4]
                    r = requests.get(
                        f"https://www.transfermarkt.com/{team_name}/startseite/verein/{team_id}?saison_id={season}",
                        headers=header)
                    soup = BeautifulSoup(r.content, "html.parser")
                    players = soup.find_all("span", class_="show-for-small")
                    for player_element in players:
                        anchor_tag = player_element.find("a")
                        if anchor_tag:
                            has_data = False
                            player_link = anchor_tag['href']
                            link_name = anchor_tag['href'].split('/')[1]
                            id = anchor_tag['href'].split('/')[4]
                            player_url = f"https://www.transfermarkt.com/{link_name}/leistungsdatendetails/spieler/{id}/plus/1?saison={season}&verein=&liga=&wettbewerb=&pos=&trainer_id="
                            player_page = requests.get(player_url, headers=header)
                            player_soup = BeautifulSoup(player_page.content, "html.parser")
                            href_list = [comps.a['href'] for comps in player_soup.select('#yw1 .no-border-links')]

                            position = player_soup.select_one(
                                '.data-header__items+ .data-header__items .data-header__label:nth-child(2) span')
                            if position:
                                position = position.text.strip()
                            else:
                                position = ''
                            compettiton_id = "IT1"
                            if position == 'Goalkeeper':
                                season_id = season
                                player_name = anchor_tag['href'].split('/')[1].replace('-', ' ').title()
                                player_id = 'p-' + anchor_tag['href'].split('/')[4]

                                if team_id:
                                    try:
                                        club_id = 'c-' + team_id
                                    except:
                                        club_id = '-'
                                apps = player_soup.select_one('.odd:nth-child(1) .zentriert:nth-child(5)')
                                if apps:
                                    try:
                                        appearances = int(apps.text)
                                    except:
                                        appearances = '-'

                                sqd = player_soup.select_one(
                                    '.odd:nth-child(1) .no-border-rechts+ .zentriert')
                                if sqd:
                                    try:
                                        squad = int(sqd.text)
                                    except:
                                        squad = '-'

                                goal = player_soup.select_one('.odd:nth-child(1) .zentriert:nth-child(7)')
                                if goal:
                                    try:
                                        goals = int(goal.text)
                                    except:
                                        goals = '-'
                                assists = '-'

                                own = player_soup.select_one('.odd:nth-child(1) .zentriert:nth-child(8)')
                                if own:
                                    try:
                                        own_goals = int(own.text)
                                    except:
                                        own_goals = 0

                                yellow = player_soup.select_one(
                                    '.odd:nth-child(1) .zentriert:nth-child(11)')
                                if yellow:
                                    try:
                                        yellow_card = int(yellow.text)
                                    except:
                                        yellow_card = 0

                                double_yellow = player_soup.select_one(
                                    '.odd:nth-child(1) .zentriert:nth-child(12)')
                                if double_yellow:
                                    try:
                                        double_yellows = int(double_yellow.text)
                                    except:
                                        double_yellows = 0

                                red = player_soup.select_one('.odd:nth-child(1) .zentriert:nth-child(13)')
                                if red:
                                    try:
                                        red_card = int(red.text)
                                    except:
                                        red_card = 0

                                goal_conced = player_soup.select_one(
                                    '.odd:nth-child(1) .zentriert:nth-child(14)')
                                if goal_conced:
                                    try:
                                        goal_conceded = int(goal_conced.text)
                                    except:
                                        goal_conceded = 0

                                clean = player_soup.select_one('.odd:nth-child(1) .zentriert:nth-child(15)')
                                if clean:
                                    try:
                                        clean_sheets = int(clean.text)
                                    except:
                                        clean_sheets = 0

                                minutes = player_soup.select_one('.odd:nth-child(1) .rechts')
                                if minutes:
                                    try:
                                        minutes_played = int(
                                            minutes.text.strip().split("'")[0].replace('.', ''))
                                    except:
                                        minutes_played = '-'

                                try:
                                    minutes_per_goal = minutes_played / goals
                                except:
                                    minutes_per_goal = '-'
                                penalty_goals = '-'

                                PPG = player_soup.select_one('.odd:nth-child(1) .zentriert:nth-child(6)')
                                if PPG:
                                    try:
                                        ponits_per_match = PPG.text
                                    except:
                                        ponits_per_match = 0

                                for counter, compet in enumerate(href_list, start=1):
                                    compet = compet.split('/')[4]
                                    if compet == 'CL' and counter == 2 and has_data == False:
                                        apps = player_soup.select_one(
                                            '.odd:nth-child(2) .zentriert:nth-child(5)')
                                        if apps:
                                            try:
                                                cl_appearances = int(apps.text)
                                            except:
                                                cl_appearances = '-'

                                        sqd = player_soup.select_one(
                                            '.odd:nth-child(2) .no-border-rechts+ .zentriert')
                                        if sqd:
                                            try:
                                                cl_squad = int(sqd.text)
                                            except:
                                                cl_squad = '-'

                                        goal = player_soup.select_one(
                                            '.odd:nth-child(2) .zentriert:nth-child(7)')
                                        if goal:
                                            try:
                                                cl_goals = int(goal.text)
                                            except:
                                                cl_goals = '-'
                                        assists = '-'

                                        own = player_soup.select_one(
                                            '.odd:nth-child(2) .zentriert:nth-child(8)')
                                        if own:
                                            try:
                                                cl_own_goals = int(own.text)
                                            except:
                                                cl_own_goals = 0

                                        yellow = player_soup.select_one(
                                            '.odd:nth-child(2) .zentriert:nth-child(11)')
                                        if yellow:
                                            try:
                                                cl_yellow_card = int(yellow.text)
                                            except:
                                                cl_yellow_card = 0

                                        double_yellow = player_soup.select_one(
                                            '.odd:nth-child(2) .zentriert:nth-child(12)')
                                        if double_yellow:
                                            try:
                                                cl_double_yellows = int(double_yellow.text)
                                            except:
                                                cl_double_yellows = 0

                                        red = player_soup.select_one(
                                            '.odd:nth-child(2) .zentriert:nth-child(13)')
                                        if red:
                                            try:
                                                cl_red_card = int(red.text)
                                            except:
                                                cl_red_card = 0

                                        goal_conced = player_soup.select_one(
                                            '.odd:nth-child(2) .zentriert:nth-child(14)')
                                        if goal_conced:
                                            try:
                                                cl_goal_conceded = int(goal_conced.text)
                                            except:
                                                cl_goal_conceded = 0

                                        clean = player_soup.select_one(
                                            '.odd:nth-child(2) .zentriert:nth-child(15)')
                                        if clean:
                                            try:
                                                clean_sheets = int(clean.text)
                                            except:
                                                clean_sheets = 0

                                        minutes = player_soup.select_one('.odd:nth-child(2) .rechts')
                                        if minutes:
                                            try:
                                                cl_minutes_played = int(
                                                    minutes.text.strip().split("'")[0].replace('.', ''))
                                            except:
                                                cl_minutes_played = '-'

                                        try:
                                            cl_minutes_per_goal = minutes_played / goals
                                        except:
                                            cl_minutes_per_goal = '-'
                                        penalty_goals = '-'

                                        PPG = player_soup.select_one(
                                            '.odd:nth-child(2) .zentriert:nth-child(6)')
                                        if PPG:
                                            try:
                                                cl_ponits_per_match = PPG.text
                                            except:
                                                cl_ponits_per_match = 0
                                        has_data = True
                                        player_data = {
                                            "Player ID": player_id,
                                            "Season ID": season_id,
                                            "Club ID": club_id,
                                            'Competition ID': compettiton_id,
                                            "Squad": squad,
                                            "Appearances": appearances,
                                            "Point Per Goal": ponits_per_match,
                                            "Goals": goals,
                                            "Assists": assists,
                                            "Own Goals": own_goals,
                                            "Yellow Cards": yellow_card,
                                            "Second Yellow Card": double_yellows,
                                            "Red Card": red_card,
                                            "Penalty Goals": penalty_goals,
                                            "Goals Conceded": goal_conceded,
                                            "Clean Sheets": clean_sheets,
                                            "Minutes Per Goal": minutes_per_goal,
                                            "Minutes Played": minutes_played,
                                            "UCL Squad": cl_squad,
                                            "UCL Appearances": cl_appearances,
                                            "UCL Point Per Goal": cl_ponits_per_match,
                                            "UCL Goals": cl_goals,
                                            "UCL Assists": assists,
                                            "UCL Own Goals": cl_own_goals,
                                            "UCL Yellow Cards": cl_yellow_card,
                                            "UCL Second Yellow Card": cl_double_yellows,
                                            "UCL Red Card": cl_red_card,
                                            "UCL Penalty Goals": penalty_goals,
                                            "UCL Goals Conceded": cl_goal_conceded,
                                            "UCL Clean Sheets": clean_sheets,
                                            "UCL Minutes Per Goal": cl_minutes_per_goal,
                                            "UCL Minutes Played": cl_minutes_played
                                        }
                                        player_data_list.append(player_data)
                                    elif compet == 'CL' and counter == 3 and has_data == False:
                                        apps = player_soup.select_one(
                                            '.even+ .odd .zentriert:nth-child(5)')
                                        if apps:
                                            try:
                                                cl_appearances = int(apps.text)
                                            except:
                                                cl_appearances = '-'

                                        sqd = player_soup.select_one(
                                            '.even+ .odd .no-border-rechts+ .zentriert')
                                        if sqd:
                                            try:
                                                cl_squad = int(sqd.text)
                                            except:
                                                cl_squad = '-'

                                        goal = player_soup.select_one(
                                            '.even+ .odd .zentriert:nth-child(7)')
                                        if goal:
                                            try:
                                                cl_goals = int(goal.text)
                                            except:
                                                cl_goals = '-'
                                        assists = '-'

                                        own = player_soup.select_one(
                                            '.even+ .odd .zentriert:nth-child(8)')
                                        if own:
                                            try:
                                                cl_own_goals = int(own.text)
                                            except:
                                                cl_own_goals = 0

                                        yellow = player_soup.select_one(
                                            '.even+ .odd .zentriert:nth-child(11)')
                                        if yellow:
                                            try:
                                                cl_yellow_card = int(yellow.text)
                                            except:
                                                cl_yellow_card = 0

                                        double_yellow = player_soup.select_one(
                                            '.even+ .odd .zentriert:nth-child(12)')
                                        if double_yellow:
                                            try:
                                                cl_double_yellows = int(double_yellow.text)
                                            except:
                                                cl_double_yellows = 0

                                        red = player_soup.select_one(
                                            '.even+ .odd .zentriert:nth-child(13)')
                                        if red:
                                            try:
                                                cl_red_card = int(red.text)
                                            except:
                                                cl_red_card = 0

                                        goal_conced = player_soup.select_one(
                                            '.even+ .odd .zentriert:nth-child(14)')
                                        if goal_conced:
                                            try:
                                                cl_goal_conceded = int(goal_conced.text)
                                            except:
                                                cl_goal_conceded = 0

                                        clean = player_soup.select_one(
                                            '.even+ .odd .zentriert:nth-child(15)')
                                        if clean:
                                            try:
                                                clean_sheets = int(clean.text)
                                            except:
                                                clean_sheets = 0

                                        minutes = player_soup.select_one('.even+ .odd .rechts')
                                        if minutes:
                                            try:
                                                cl_minutes_played = int(
                                                    minutes.text.strip().split("'")[0].replace('.', ''))
                                            except:
                                                cl_minutes_played = '-'

                                        try:
                                            cl_minutes_per_goal = minutes_played / goals
                                        except:
                                            cl_minutes_per_goal = '-'
                                        penalty_goals = '-'

                                        PPG = player_soup.select_one(
                                            '.even+ .odd .zentriert:nth-child(6)')
                                        if PPG:
                                            try:
                                                cl_ponits_per_match = PPG.text
                                            except:
                                                cl_ponits_per_match = 0
                                        has_data = True
                                        player_data = {
                                            "Player ID": player_id,
                                            "Season ID": season_id,
                                            "Club ID": club_id,
                                            'Competition ID': compettiton_id,
                                            "Squad": squad,
                                            "Appearances": appearances,
                                            "Point Per Goal": ponits_per_match,
                                            "Goals": goals,
                                            "Assists": assists,
                                            "Own Goals": own_goals,
                                            "Yellow Cards": yellow_card,
                                            "Second Yellow Card": double_yellows,
                                            "Red Card": red_card,
                                            "Penalty Goals": penalty_goals,
                                            "Goals Conceded": goal_conceded,
                                            "Clean Sheets": clean_sheets,
                                            "Minutes Per Goal": minutes_per_goal,
                                            "Minutes Played": minutes_played,
                                            "UCL Squad": cl_squad,
                                            "UCL Appearances": cl_appearances,
                                            "UCL Point Per Goal": cl_ponits_per_match,
                                            "UCL Goals": cl_goals,
                                            "UCL Assists": assists,
                                            "UCL Own Goals": cl_own_goals,
                                            "UCL Yellow Cards": cl_yellow_card,
                                            "UCL Second Yellow Card": cl_double_yellows,
                                            "UCL Red Card": cl_red_card,
                                            "UCL Penalty Goals": penalty_goals,
                                            "UCL Goals Conceded": cl_goal_conceded,
                                            "UCL Clean Sheets": clean_sheets,
                                            "UCL Minutes Per Goal": cl_minutes_per_goal,
                                            "UCL Minutes Played": cl_minutes_played
                                        }
                                        player_data_list.append(player_data)
                                    elif compet == 'CL' and counter == 4 and has_data == False:
                                        apps = player_soup.select_one(
                                            '.even~ .even .zentriert:nth-child(5)')
                                        if apps:
                                            try:
                                                cl_appearances = int(apps.text)
                                            except:
                                                cl_appearances = '-'

                                        sqd = player_soup.select_one(
                                            '.even~ .even .no-border-rechts+ .zentriert')
                                        if sqd:
                                            try:
                                                cl_squad = int(sqd.text)
                                            except:
                                                cl_squad = '-'

                                        goal = player_soup.select_one(
                                            '.even~ .even .zentriert:nth-child(7)')
                                        if goal:
                                            try:
                                                cl_goals = int(goal.text)
                                            except:
                                                cl_goals = '-'
                                        assists = '-'

                                        own = player_soup.select_one(
                                            '.even~ .even .zentriert:nth-child(8)')
                                        if own:
                                            try:
                                                cl_own_goals = int(own.text)
                                            except:
                                                cl_own_goals = 0

                                        yellow = player_soup.select_one(
                                            '.even~ .even .zentriert:nth-child(11)')
                                        if yellow:
                                            try:
                                                cl_yellow_card = int(yellow.text)
                                            except:
                                                cl_yellow_card = 0

                                        double_yellow = player_soup.select_one(
                                            '.even~ .even .zentriert:nth-child(12)')
                                        if double_yellow:
                                            try:
                                                cl_double_yellows = int(double_yellow.text)
                                            except:
                                                cl_double_yellows = 0

                                        red = player_soup.select_one(
                                            '.even~ .even .zentriert:nth-child(13)')
                                        if red:
                                            try:
                                                cl_red_card = int(red.text)
                                            except:
                                                cl_red_card = 0

                                        goal_conced = player_soup.select_one(
                                            '.even~ .even .zentriert:nth-child(14)')
                                        if goal_conced:
                                            try:
                                                cl_goal_conceded = int(goal_conced.text)
                                            except:
                                                cl_goal_conceded = 0

                                        clean = player_soup.select_one(
                                            '.even~ .even .zentriert:nth-child(15)')
                                        if clean:
                                            try:
                                                clean_sheets = int(clean.text)
                                            except:
                                                clean_sheets = 0

                                        minutes = player_soup.select_one('.even~ .even .rechts')
                                        if minutes:
                                            try:
                                                cl_minutes_played = int(
                                                    minutes.text.strip().split("'")[0].replace('.', ''))
                                            except:
                                                cl_minutes_played = '-'

                                        try:
                                            cl_minutes_per_goal = minutes_played / goals
                                        except:
                                            cl_minutes_per_goal = '-'
                                        penalty_goals = '-'

                                        PPG = player_soup.select_one(
                                            '.even~ .even .zentriert:nth-child(6)')
                                        if PPG:
                                            try:
                                                cl_ponits_per_match = PPG.text
                                            except:
                                                cl_ponits_per_match = 0
                                        has_data = True
                                        player_data = {
                                            "Player ID": player_id,
                                            "Season ID": season_id,
                                            "Club ID": club_id,
                                            'Competition ID': compettiton_id,
                                            "Squad": squad,
                                            "Appearances": appearances,
                                            "Point Per Goal": ponits_per_match,
                                            "Goals": goals,
                                            "Assists": assists,
                                            "Own Goals": own_goals,
                                            "Yellow Cards": yellow_card,
                                            "Second Yellow Card": double_yellows,
                                            "Red Card": red_card,
                                            "Penalty Goals": penalty_goals,
                                            "Goals Conceded": goal_conceded,
                                            "Clean Sheets": clean_sheets,
                                            "Minutes Per Goal": minutes_per_goal,
                                            "Minutes Played": minutes_played,
                                            "UCL Squad": cl_squad,
                                            "UCL Appearances": cl_appearances,
                                            "UCL Point Per Goal": cl_ponits_per_match,
                                            "UCL Goals": cl_goals,
                                            "UCL Assists": assists,
                                            "UCL Own Goals": cl_own_goals,
                                            "UCL Yellow Cards": cl_yellow_card,
                                            "UCL Second Yellow Card": cl_double_yellows,
                                            "UCL Red Card": cl_red_card,
                                            "UCL Penalty Goals": penalty_goals,
                                            "UCL Goals Conceded": cl_goal_conceded,
                                            "UCL Clean Sheets": clean_sheets,
                                            "UCL Minutes Per Goal": cl_minutes_per_goal,
                                            "UCL Minutes Played": cl_minutes_played
                                        }
                                        player_data_list.append(player_data)
                                    elif has_data == False:
                                        cl_squad = '-'
                                        cl_appearances = '-'
                                        cl_ponits_per_match = '-'
                                        cl_goals = '-'
                                        cl_own_goals = '-'
                                        cl_yellow_card = '-'
                                        cl_double_yellows = '-'
                                        cl_red_card = '-'
                                        cl_goal_conceded = '-'
                                        cl_minutes_per_goal = '-'
                                        cl_minutes_played = '-'
                                        player_data = {
                                            "Player ID": player_id,
                                            "Season ID": season_id,
                                            "Club ID": club_id,
                                            'Competition ID': compettiton_id,
                                            "Squad": squad,
                                            "Appearances": appearances,
                                            "Point Per Goal": ponits_per_match,
                                            "Goals": goals,
                                            "Assists": assists,
                                            "Own Goals": own_goals,
                                            "Yellow Cards": yellow_card,
                                            "Second Yellow Card": double_yellows,
                                            "Red Card": red_card,
                                            "Penalty Goals": penalty_goals,
                                            "Goals Conceded": goal_conceded,
                                            "Clean Sheets": clean_sheets,
                                            "Minutes Per Goal": minutes_per_goal,
                                            "Minutes Played": minutes_played,
                                            "UCL Squad": cl_squad,
                                            "UCL Appearances": cl_appearances,
                                            "UCL Point Per Goal": cl_ponits_per_match,
                                            "UCL Goals": cl_goals,
                                            "UCL Assists": assists,
                                            "UCL Own Goals": cl_own_goals,
                                            "UCL Yellow Cards": cl_yellow_card,
                                            "UCL Second Yellow Card": cl_double_yellows,
                                            "UCL Red Card": cl_red_card,
                                            "UCL Penalty Goals": penalty_goals,
                                            "UCL Goals Conceded": cl_goal_conceded,
                                            "UCL Clean Sheets": clean_sheets,
                                            "UCL Minutes Per Goal": cl_minutes_per_goal,
                                            "UCL Minutes Played": cl_minutes_played
                                        }
                                        player_data_list.append(player_data)

                                    count += 1
                                    print(count)
                                    if count == 16000:
                                        break
                            elif position == '':
                                continue
                            elif position != 'Goalkeeper':
                                season_id = season
                                player_name = anchor_tag['href'].split('/')[1].replace('-', ' ').title()
                                player_id = 'p-' + anchor_tag['href'].split('/')[4]
                                if team_id:
                                    try:
                                        club_id = 'c-' + team_id
                                    except:
                                        club_id = '-'
                                apps = player_soup.select_one('.odd:nth-child(1) .zentriert:nth-child(5)')
                                if apps:
                                    try:
                                        appearances = int(apps.text)
                                    except:
                                        appearances = '-'

                                sqd = player_soup.select_one(
                                    '.odd:nth-child(1) .no-border-rechts+ .zentriert')
                                if sqd:
                                    try:
                                        squad = int(sqd.text)
                                    except:
                                        squad = '-'
                                goal = player_soup.select_one('.odd:nth-child(1) .zentriert:nth-child(7)')
                                if goal:
                                    try:
                                        goals = int(goal.text)
                                    except:
                                        goals = 0

                                asist = player_soup.select_one('.odd:nth-child(1) .zentriert:nth-child(8)')
                                if yellow:
                                    try:
                                        assists = int(asist.text)
                                    except:
                                        assists = 0
                                own = player_soup.select_one('.odd:nth-child(1) .zentriert:nth-child(9)')
                                if own:
                                    try:
                                        own_goals = int(own.text)
                                    except:
                                        own_goals = 0
                                yellow = player_soup.select_one(
                                    '.odd:nth-child(1) .zentriert:nth-child(12)')
                                if yellow:
                                    try:
                                        yellow_card = int(yellow.text)
                                    except:
                                        yellow_card = 0

                                double_yellow = player_soup.select_one(
                                    '.odd:nth-child(1) .zentriert:nth-child(13)')
                                if double_yellow:
                                    try:
                                        double_yellows = int(double_yellow.text)
                                    except:
                                        double_yellows = 0

                                red = player_soup.select_one('.odd:nth-child(1) .zentriert:nth-child(14)')
                                if red:
                                    try:
                                        red_card = int(red.text)
                                    except:
                                        red_card = 0

                                goal_conceded = '-'
                                clean_sheets = '-'

                                peng = player_soup.select_one('.odd:nth-child(1) .zentriert:nth-child(15)')
                                if red:
                                    try:
                                        penalty_goals = int(peng.text)
                                    except:
                                        penalty_goals = 0

                                minutes = player_soup.select_one('.odd:nth-child(1) .rechts+ .rechts')
                                if minutes:
                                    try:
                                        minutes_played = int(
                                            minutes.text.strip().split("'")[0].replace('.', ''))
                                    except:
                                        minutes_played = '-'

                                minutes_pg = player_soup.select_one('.odd:nth-child(1) .zentriert+ .rechts')
                                if minutes:
                                    try:
                                        minutes_per_goal = int(minutes_pg.text.strip().split("'")[0])
                                    except:
                                        minutes_per_goal = '-'

                                PPG = player_soup.select_one('.odd:nth-child(1) .zentriert:nth-child(6)')
                                if PPG:
                                    try:
                                        ponits_per_match = PPG.text
                                    except:
                                        ponits_per_match = 0
                                for counter, compet in enumerate(href_list, start=1):
                                    compet = compet.split('/')[4]
                                    if compet == 'CL' and counter == 2 and has_data == False:
                                        apps = player_soup.select_one(
                                            '.odd:nth-child(2) .zentriert:nth-child(5)')
                                        if apps:
                                            try:
                                                cl_appearances = int(apps.text)
                                            except:
                                                cl_appearances = '-'

                                        sqd = player_soup.select_one(
                                            '.odd:nth-child(2) .no-border-rechts+ .zentriert')
                                        if sqd:
                                            try:
                                                cl_squad = int(sqd.text)
                                            except:
                                                cl_squad = '-'
                                        goal = player_soup.select_one(
                                            '.odd:nth-child(2) .zentriert:nth-child(7)')
                                        if goal:
                                            try:
                                                cl_goals = int(goal.text)
                                            except:
                                                cl_goals = 0

                                        asist = player_soup.select_one(
                                            '.odd:nth-child(2) .zentriert:nth-child(8)')
                                        if yellow:
                                            try:
                                                cl_assists = int(asist.text)
                                            except:
                                                cl_assists = 0
                                        own = player_soup.select_one(
                                            '.odd:nth-child(2) .zentriert:nth-child(9)')
                                        if own:
                                            try:
                                                cl_own_goals = int(own.text)
                                            except:
                                                cl_own_goals = 0
                                        yellow = player_soup.select_one(
                                            '.odd:nth-child(2) .zentriert:nth-child(12)')
                                        if yellow:
                                            try:
                                                cl_yellow_card = int(yellow.text)
                                            except:
                                                cl_yellow_card = 0

                                        double_yellow = player_soup.select_one(
                                            '.odd:nth-child(2) .zentriert:nth-child(13)')
                                        if double_yellow:
                                            try:
                                                cl_double_yellows = int(double_yellow.text)
                                            except:
                                                cl_double_yellows = 0

                                        red = player_soup.select_one(
                                            '.odd:nth-child(2) .zentriert:nth-child(14)')
                                        if red:
                                            try:
                                                cl_red_card = int(red.text)
                                            except:
                                                cl_red_card = 0

                                        goal_conceded = '-'
                                        clean_sheets = '-'

                                        peng = player_soup.select_one(
                                            '.odd:nth-child(2) .zentriert:nth-child(15)')
                                        if red:
                                            try:
                                                cl_penalty_goals = int(peng.text)
                                            except:
                                                cl_penalty_goals = 0

                                        minutes = player_soup.select_one('.odd:nth-child(2) .rechts+ .rechts')
                                        if minutes:
                                            try:
                                                cl_minutes_played = int(
                                                    minutes.text.strip().split("'")[0].replace('.', ''))
                                            except:
                                                cl_minutes_played = '-'

                                        minutes_pg = player_soup.select_one(
                                            '.odd:nth-child(2) .zentriert+ .rechts')
                                        if minutes:
                                            try:
                                                cl_minutes_per_goal = int(minutes_pg.text.strip().split("'")[0])
                                            except:
                                                cl_minutes_per_goal = '-'

                                        PPG = player_soup.select_one(
                                            '.odd:nth-child(2) .zentriert:nth-child(6)')
                                        if PPG:
                                            try:
                                                cl_ponits_per_match = PPG.text
                                            except:
                                                cl_ponits_per_match = 0
                                        has_data = True
                                        player_data = {
                                            "Player ID": player_id,
                                            "Season ID": season_id,
                                            "Club ID": club_id,
                                            'Competition ID': compettiton_id,
                                            "Squad": squad,
                                            "Appearances": appearances,
                                            "Point Per Goal": ponits_per_match,
                                            "Goals": goals,
                                            "Assists": assists,
                                            "Own Goals": own_goals,
                                            "Yellow Cards": yellow_card,
                                            "Second Yellow Card": double_yellows,
                                            "Red Card": red_card,
                                            "Penalty Goals": penalty_goals,
                                            "Goals Conceded": goal_conceded,
                                            "Clean Sheets": clean_sheets,
                                            "Minutes Per Goal": minutes_per_goal,
                                            "Minutes Played": minutes_played,
                                            "UCL Squad": cl_squad,
                                            "UCL Appearances": cl_appearances,
                                            "UCL Point Per Goal": cl_ponits_per_match,
                                            "UCL Goals": cl_goals,
                                            "UCL Assists": cl_assists,
                                            "UCL Own Goals": cl_own_goals,
                                            "UCL Yellow Cards": cl_yellow_card,
                                            "UCL Second Yellow Card": cl_double_yellows,
                                            "UCL Red Card": cl_red_card,
                                            "UCL Penalty Goals": cl_penalty_goals,
                                            "UCL Goals Conceded": cl_goal_conceded,
                                            "UCL Clean Sheets": clean_sheets,
                                            "UCL Minutes Per Goal": cl_minutes_per_goal,
                                            "UCL Minutes Played": cl_minutes_played
                                        }
                                        player_data_list.append(player_data)
                                    elif compet == 'CL' and counter == 3 and has_data == False:
                                        apps = player_soup.select_one(
                                            '.even+ .odd .zentriert:nth-child(5)')
                                        if apps:
                                            try:
                                                cl_appearances = int(apps.text)
                                            except:
                                                cl_appearances = '-'

                                        sqd = player_soup.select_one(
                                            '.even+ .odd .no-border-rechts+ .zentriert')
                                        if sqd:
                                            try:
                                                cl_squad = int(sqd.text)
                                            except:
                                                cl_squad = '-'
                                        goal = player_soup.select_one(
                                            '.even+ .odd .zentriert:nth-child(7)')
                                        if goal:
                                            try:
                                                cl_goals = int(goal.text)
                                            except:
                                                cl_goals = 0

                                        asist = player_soup.select_one(
                                            '.even+ .odd .zentriert:nth-child(8)')
                                        if yellow:
                                            try:
                                                cl_assists = int(asist.text)
                                            except:
                                                cl_assists = 0
                                        own = player_soup.select_one(
                                            '.even+ .odd .zentriert:nth-child(9)')
                                        if own:
                                            try:
                                                cl_own_goals = int(own.text)
                                            except:
                                                cl_own_goals = 0
                                        yellow = player_soup.select_one(
                                            '.even+ .odd .zentriert:nth-child(12)')
                                        if yellow:
                                            try:
                                                cl_yellow_card = int(yellow.text)
                                            except:
                                                cl_yellow_card = 0

                                        double_yellow = player_soup.select_one(
                                            '.even+ .odd .zentriert:nth-child(13)')
                                        if double_yellow:
                                            try:
                                                cl_double_yellows = int(double_yellow.text)
                                            except:
                                                cl_double_yellows = 0

                                        red = player_soup.select_one(
                                            '.even+ .odd .zentriert:nth-child(14)')
                                        if red:
                                            try:
                                                cl_red_card = int(red.text)
                                            except:
                                                cl_red_card = 0

                                        goal_conceded = '-'
                                        clean_sheets = '-'

                                        peng = player_soup.select_one(
                                            '.even+ .odd .zentriert:nth-child(15)')
                                        if red:
                                            try:
                                                cl_penalty_goals = int(peng.text)
                                            except:
                                                cl_penalty_goals = 0

                                        minutes = player_soup.select_one('.even+ .odd .rechts+ .rechts')
                                        if minutes:
                                            try:
                                                cl_minutes_played = int(
                                                    minutes.text.strip().split("'")[0].replace('.', ''))
                                            except:
                                                cl_minutes_played = '-'

                                        minutes_pg = player_soup.select_one(
                                            '.even+ .odd .zentriert+ .rechts')
                                        if minutes:
                                            try:
                                                cl_minutes_per_goal = int(minutes_pg.text.strip().split("'")[0])
                                            except:
                                                cl_minutes_per_goal = '-'

                                        PPG = player_soup.select_one(
                                            '.even+ .odd .zentriert:nth-child(6)')
                                        if PPG:
                                            try:
                                                cl_ponits_per_match = PPG.text
                                            except:
                                                cl_ponits_per_match = 0
                                        has_data = True
                                        player_data = {
                                            "Player ID": player_id,
                                            "Season ID": season_id,
                                            "Club ID": club_id,
                                            'Competition ID': compettiton_id,
                                            "Squad": squad,
                                            "Appearances": appearances,
                                            "Point Per Goal": ponits_per_match,
                                            "Goals": goals,
                                            "Assists": assists,
                                            "Own Goals": own_goals,
                                            "Yellow Cards": yellow_card,
                                            "Second Yellow Card": double_yellows,
                                            "Red Card": red_card,
                                            "Penalty Goals": penalty_goals,
                                            "Goals Conceded": goal_conceded,
                                            "Clean Sheets": clean_sheets,
                                            "Minutes Per Goal": minutes_per_goal,
                                            "Minutes Played": minutes_played,
                                            "UCL Squad": cl_squad,
                                            "UCL Appearances": cl_appearances,
                                            "UCL Point Per Goal": cl_ponits_per_match,
                                            "UCL Goals": cl_goals,
                                            "UCL Assists": cl_assists,
                                            "UCL Own Goals": cl_own_goals,
                                            "UCL Yellow Cards": cl_yellow_card,
                                            "UCL Second Yellow Card": cl_double_yellows,
                                            "UCL Red Card": cl_red_card,
                                            "UCL Penalty Goals": cl_penalty_goals,
                                            "UCL Goals Conceded": cl_goal_conceded,
                                            "UCL Clean Sheets": clean_sheets,
                                            "UCL Minutes Per Goal": cl_minutes_per_goal,
                                            "UCL Minutes Played": cl_minutes_played
                                        }
                                        player_data_list.append(player_data)
                                    elif compet == 'CL' and counter == 4 and has_data == False:
                                        apps = player_soup.select_one(
                                            '.even~ .even .zentriert:nth-child(5)')
                                        if apps:
                                            try:
                                                cl_appearances = int(apps.text)
                                            except:
                                                cl_appearances = '-'

                                        sqd = player_soup.select_one(
                                            '.even~ .even .no-border-rechts+ .zentriert')
                                        if sqd:
                                            try:
                                                cl_squad = int(sqd.text)
                                            except:
                                                cl_squad = '-'
                                        goal = player_soup.select_one(
                                            '.even~ .even .zentriert:nth-child(7)')
                                        if goal:
                                            try:
                                                cl_goals = int(goal.text)
                                            except:
                                                cl_goals = 0

                                        asist = player_soup.select_one(
                                            '.even~ .even .zentriert:nth-child(8)')
                                        if yellow:
                                            try:
                                                cl_assists = int(asist.text)
                                            except:
                                                cl_assists = 0
                                        own = player_soup.select_one(
                                            '.even~ .even .zentriert:nth-child(9)')
                                        if own:
                                            try:
                                                cl_own_goals = int(own.text)
                                            except:
                                                cl_own_goals = 0
                                        yellow = player_soup.select_one(
                                            '.even~ .even .zentriert:nth-child(12)')
                                        if yellow:
                                            try:
                                                cl_yellow_card = int(yellow.text)
                                            except:
                                                cl_yellow_card = 0

                                        double_yellow = player_soup.select_one(
                                            '.even~ .even .zentriert:nth-child(13)')
                                        if double_yellow:
                                            try:
                                                cl_double_yellows = int(double_yellow.text)
                                            except:
                                                cl_double_yellows = 0

                                        red = player_soup.select_one(
                                            '.even~ .even .zentriert:nth-child(14)')
                                        if red:
                                            try:
                                                cl_red_card = int(red.text)
                                            except:
                                                cl_red_card = 0

                                        goal_conceded = '-'
                                        clean_sheets = '-'

                                        peng = player_soup.select_one(
                                            '.even~ .even .zentriert:nth-child(15)')
                                        if red:
                                            try:
                                                cl_penalty_goals = int(peng.text)
                                            except:
                                                cl_penalty_goals = 0

                                        minutes = player_soup.select_one('.even~ .even .rechts+ .rechts')
                                        if minutes:
                                            try:
                                                cl_minutes_played = int(
                                                    minutes.text.strip().split("'")[0].replace('.', ''))
                                            except:
                                                cl_minutes_played = '-'

                                        minutes_pg = player_soup.select_one(
                                            '.even~ .even .zentriert+ .rechts')
                                        if minutes:
                                            try:
                                                cl_minutes_per_goal = int(minutes_pg.text.strip().split("'")[0])
                                            except:
                                                cl_minutes_per_goal = '-'

                                        PPG = player_soup.select_one(
                                            '.even~ .even .zentriert:nth-child(6)')
                                        if PPG:
                                            try:
                                                cl_ponits_per_match = PPG.text
                                            except:
                                                cl_ponits_per_match = 0
                                        has_data = True
                                        player_data = {
                                            "Player ID": player_id,
                                            "Season ID": season_id,
                                            "Club ID": club_id,
                                            'Competition ID': compettiton_id,
                                            "Squad": squad,
                                            "Appearances": appearances,
                                            "Point Per Goal": ponits_per_match,
                                            "Goals": goals,
                                            "Assists": assists,
                                            "Own Goals": own_goals,
                                            "Yellow Cards": yellow_card,
                                            "Second Yellow Card": double_yellows,
                                            "Red Card": red_card,
                                            "Penalty Goals": penalty_goals,
                                            "Goals Conceded": goal_conceded,
                                            "Clean Sheets": clean_sheets,
                                            "Minutes Per Goal": minutes_per_goal,
                                            "Minutes Played": minutes_played,
                                            "UCL Squad": cl_squad,
                                            "UCL Appearances": cl_appearances,
                                            "UCL Point Per Goal": cl_ponits_per_match,
                                            "UCL Goals": cl_goals,
                                            "UCL Assists": cl_assists,
                                            "UCL Own Goals": cl_own_goals,
                                            "UCL Yellow Cards": cl_yellow_card,
                                            "UCL Second Yellow Card": cl_double_yellows,
                                            "UCL Red Card": cl_red_card,
                                            "UCL Penalty Goals": cl_penalty_goals,
                                            "UCL Goals Conceded": cl_goal_conceded,
                                            "UCL Clean Sheets": clean_sheets,
                                            "UCL Minutes Per Goal": cl_minutes_per_goal,
                                            "UCL Minutes Played": cl_minutes_played
                                        }
                                        player_data_list.append(player_data)
                                    elif has_data == False:
                                        cl_squad = '-'
                                        cl_appearances = '-'
                                        cl_ponits_per_match = '-'
                                        cl_goals = '-'
                                        cl_assists = '-'
                                        cl_own_goals = '-'
                                        cl_yellow_card = '-'
                                        cl_double_yellows = '-'
                                        cl_red_card = '-'
                                        cl_penalty_goals = '-'
                                        cl_goal_conceded = '-'
                                        cl_minutes_per_goal = '-'
                                        cl_minutes_played = '-'
                                        player_data = {
                                            "Player ID": player_id,
                                            "Season ID": season_id,
                                            "Club ID": club_id,
                                            'Competition ID': compettiton_id,
                                            "Squad": squad,
                                            "Appearances": appearances,
                                            "Point Per Goal": ponits_per_match,
                                            "Goals": goals,
                                            "Assists": assists,
                                            "Own Goals": own_goals,
                                            "Yellow Cards": yellow_card,
                                            "Second Yellow Card": double_yellows,
                                            "Red Card": red_card,
                                            "Penalty Goals": penalty_goals,
                                            "Goals Conceded": goal_conceded,
                                            "Clean Sheets": clean_sheets,
                                            "Minutes Per Goal": minutes_per_goal,
                                            "Minutes Played": minutes_played,
                                            "UCL Squad": cl_squad,
                                            "UCL Appearances": cl_appearances,
                                            "UCL Point Per Goal": cl_ponits_per_match,
                                            "UCL Goals": cl_goals,
                                            "UCL Assists": cl_assists,
                                            "UCL Own Goals": cl_own_goals,
                                            "UCL Yellow Cards": cl_yellow_card,
                                            "UCL Second Yellow Card": cl_double_yellows,
                                            "UCL Red Card": cl_red_card,
                                            "UCL Penalty Goals": cl_penalty_goals,
                                            "UCL Goals Conceded": cl_goal_conceded,
                                            "UCL Clean Sheets": clean_sheets,
                                            "UCL Minutes Per Goal": cl_minutes_per_goal,
                                            "UCL Minutes Played": cl_minutes_played
                                        }
                                        player_data_list.append(player_data)

                                    print(count)
                                    count += 1
                                    if count == 16000:
                                        break

        except Exception as e:
            # Handle the exception or error
            print("An error occurred:", str(e))
        # Optionally, you can log the error or perform other actions

        else:
            # Code to execute when internet is not available
            time.sleep(30)
        if count>16000:
            break